## Processing video data for FaceNet and Landmark model

Both FaceNet and Landmark model use 160x160 videos so their data can be processed together

In [ ]:
import dlib
import numpy as np
import skvideo.io
import os
from video_processes import *
import pickle

### Load dlib facial detector and predictor

In [ ]:
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

<h3> Processing Videos </h3>

Videos should be in folders by classes. Result includes:
1. a list of processed videos (usable for FaceNet due to the 160x160 size)
2. facial landmark and eye data for each frame
3. label data

In [ ]:
####
#parameters
folder_list = ["Focus", "Distract"] #list of video folders. folder name will be used as labels
resample_center_step = 12 #step to skip frames for facial center and max distances to boundary
resample_crop_step = 2 #step to skip frames for storing after cropping
cropped_size = 160 #size of cropped square video

####
#functional codes
pr_videos = []
landmark_and_eye = []
labels = []

for folder in folder_list:
    #get video list
    allfiles = os.listdir(folder+'/')
    #list to store processed videos
    for fn in allfiles:
        try: #try open videos
            v = skvideo.io.vread(folder + '/' + fn)
            H, W = v.shape[1:-1]   #video dimension
        except:
            continue
        #roughly query facial center and max distance to boundaries in videos
        centers = []
        mds = []
        processed_video = []
        for f in v[::resample_center_step]:
            try:
                c, md = get_face_region(f, detector, predictor, 20)
                centers.append(c)
                mds.append(md)
            except:
                continue
        #using median center and max of max distances
        C = np.int32(np.median(np.array(centers),axis=0))
        M = np.int32(np.max(mds))
        #resample and crop to square videos
        #also extract facial landmarks
        video_landmarks = []
        for frame in v[::resample_crop_step]:
            cropped_frame = squarize_frame(frame, C, M, (cropped_size,cropped_size))
            processed_video.append(cropped_frame)
            #extract landmark of current frame
            rect = detector(cropped_frame, 0) [0]
            shape = predictor(gray, rect)
            try:
                shape_np = shape_to_np(shape)
            except:
                shape_np = np.zeros(shape_np.shape)
            video_landmarks.append(shape_np)
        processed_video = np.array(processed_video, dtype=np.uint8)
        #extract eye data of current video
        video_eye_info = extract_eye_info(processed_video)
        #store processed video
        pr_videos.append(processed_video)
        #store landmarks and eye data
        landmark_and_eye.append(np.hstack([video_landmarks, video_eye_info]))
        #store label
        labels.append(folder)

### Save video data and labels

In [ ]:
of = open('processed_videos_160.obj', 'wb')
pickle.dump([pr_videos, labels], of)
of.close()

### Save landmarks and eye data and labels

In [ ]:
of = open('landmark_eye_data.obj', 'wb')
pickle.dump([landmark_and_eye, labels], of)
of.close()